# Setup
- Install dependencies
- Import packages
- To run with the Spotify API, you need the Kaggle secrets `spotify_client_secret` and `spotify_client_id`

In [1]:
!pip install spotipy seaborn

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from kaggle_secrets import UserSecretsClient

spotify_client_secret = UserSecretsClient().get_secret('spotify_client_secret')
spotify_client_id = UserSecretsClient().get_secret('spotify_client_id')

In [4]:
charts = pd.read_csv('/kaggle/input/spotify-charts/charts.csv')

In [5]:
analysis = pd.read_csv('/kaggle/input/spotifygermanchartsanalysis/analysis.csv')

In [6]:
charts.shape

# Exploring the Characteristics of Songs in Charts over time


Spotify offers charts, updated in three-day intervals. In this notebook, we're going to explore what trends there are in the charts regarding the features, meaning characteristics like "acousticness", "vocals", ..., of the music over time. For this we are using two datasets.
1. The Spotify-Charts dataset found on kaggle https://www.kaggle.com/dhruvildave/spotify-charts
2. A dataset of characteristics for each song. Because I didn't find a good enough dataset, we first filter the songs listed in the Spotify-Charts dataset to unique songs that we need and then use the Spotify API audio-features endpoint (https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features)

## Spotify-Charts Dataset
The Spotify-Charts dataset has only one csv-file, so all data is present in tabular structure. It contains one song per row. Each row has the attributes title, rank, date, artist, url, region, chart, trend and streams. 

The attributes `title`, `artist`, `url` and `streams` describe the track being present in a playlist at the given `date`. Artist can be seen as multi-categorical data, however multiple artists are concatenated with a comma, so that some preprocessing would be required to use it as such. Streams are the absolute number of streams, meaning "listening sessions". 

Each row describes the playlist the track is in with the attributes `chart` and `region`, where chart is categorical and either `top200` or `viral50` - the two playlists present in the dataset. Region is also categorical, it contains 70 different region names. The attributes `rank` and `trend` describe the position of the track, respectively the change in the position. `rank` contains values between 1 and 200 for each set of tracks in `top200` (meaning rows with same chart, region and date) or just 1 to 50 for tracks in `viral50`.

Because of Spotify's API query limitation, we'll only use the German `viral50` for now, so we filter the rows by `region="Germany"` and `chart="viral50"` 

In [7]:
# Available regions in the dataset
charts['region'].unique()

In [8]:
# Filter charts to said region and chart
germanViral50 = charts[(charts['region']=='Germany') & (charts['chart']=='viral50')]
germanViral50.info()

In [9]:
# Find unique tracks in the charts
uniqueTracks = germanViral50['url'].unique()
uniqueTracks.shape

### Top 200
We can do the same filtering & visualizations on the top200 charts, however as I did not find noticeable differences except for longer execution times, I will omit them.

In [12]:
top200 = charts[(charts['region']=='Germany') & (charts['chart']=='top200')]
top200.info()
uniqueTop200 = top200['url'].unique()
uniqueTop200.shape

## Retrieving Spotify Track Features
We query the api in batches of 100, which is the query limit of Spotify.

In [ ]:
auth_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Helper function to split sequence in to chunks of "size" 
# Src: https://stackoverflow.com/questions/434287/what-is-the-most-pythonic-way-to-iterate-over-a-list-in-chunks/434328#434328
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [ ]:
result_list = []
for urls in chunker(uniqueTop200, 100):
    req_result = sp.audio_features(tracks=urls)
    result_list += req_result

In [ ]:
# There are very few tracks (I found ~1 for each chart) which don't have an analysis. They'll be dropped
# when merging the tables

# Remove empty entries
result_list = list(filter(lambda x: x is not None, result_list))

In [ ]:
analysis = pd.DataFrame(result_list)
analysis.head()

In [ ]:
# Store it as CSV for uploading to my own "dataset": https://www.kaggle.com/franzsw/spotifygermanchartsanalysis
analysis.to_csv('/kaggle/working/analysis.csv')

## Visualizing the Data
To make visualizations easier we first join the two tables on `germanViral50.url == analysis.track_href`

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme()
# Set default figure size
sns.set(rc={'figure.figsize':(12,8.27)})

## Merge the tables
First, we extract the trackId for each chart entry from the url. Then we merge them on the respective track id attribute.

In [15]:
germanViral50['trackId'] = germanViral50['url'].apply(lambda url: url.replace('https://open.spotify.com/track/', ''))
merged_table = germanViral50.merge(analysis, left_on='trackId', right_on='id')
merged_table.head()

In [16]:
# Prepare merged table with additional data and type conversion
# For easier filtering / grouping, we add a year, month and day column (of integers)
merged_table[['year', 'month', 'day']] = merged_table['date'].str.split('-', n=3, expand=True).astype('int')
# make the "key" value categorical
merged_table['key'] = merged_table['key'].astype('category')
merged_table.info()

In [38]:
# November-December 2021 entries don't seem to be complete by now / missing, so we'll ignore them
merged_table = merged_table[(merged_table['year'] != 2021) | ((merged_table['year'] == 2021) & (merged_table['month'] <= 10))]
# Sorting the entries by year & month already gives them the order we want in our plots.
# This way, we can leave out additional ordering in the plots.
merged_table = merged_table.sort_values(['year', 'month'])

In [41]:
# Additional filtering
# E.g. we can filter the data to be of a specific rank, ...
# merged_table = merged_table[merged_table['rank'] <= 5]

In [25]:
# Snippet to output the (calculated) correlation between two features
merged_table['valence'].corr(merged_table['energy'])

In [27]:
# Labels for the X-Axis... Next time with a loop :D
xlabels = [
    '17-01','17-02','17-03','17-04','17-05','17-06','17-07','17-08','17-09','17-10','17-11','17-12',
    '18-01','18-02','18-03','18-04','18-05','18-06','18-07','18-08','18-09','18-10','18-11','18-12',
    '19-01','19-02','19-03','19-04','19-05','19-06','19-07','19-08','19-09','19-10','19-11','19-12',
    '20-01','20-02','20-03','20-04','20-05','20-06','20-07','20-08','20-09','20-10','20-11','20-12',
    '21-01','21-02','21-03','21-04','21-05','21-06','21-07','21-08','21-09','21-10'
          ]
xlabels_clean = [
    '2017-01','02','03','04','05','06','07','08','09','10','11','12',
    '2018-01','02','03','04','05','06','07','08','09','10','11','12',
    '2019-01','02','03','04','05','06','07','08','09','10','11','12',
    '2020-01','02','03','04','05','06','07','08','09','10','11','12',
    '2021-01','02','03','04','05','06','07','08','09','10'
          ]

## Pairplot
Pairplot between selected features to see correlation and distribution

In [ ]:
sns.pairplot(merged_table, vars=['energy', 'valence', 'loudness', 'liveness', 'speechiness', 'danceability'], kind='hist', diag_kind="hist", diag_kws={'bins': 25} , plot_kws={'bins': 25} )

## Keys used in songs

In [43]:
# Visualize keys in a heatmap
# First count all keys
keyCounts = merged_table.groupby(by=['year', 'month', 'key'])['id'].count()
# We want to display relative frequency per month (each month summed up should theoretically be the same color)
keyCountRelativePerMonth = keyCounts.groupby(by=['year', 'month']).transform(lambda x: x / max(x.sum(), 1))
# Shape them into a (month, keys) array - there are 12 keys
keyCountPerMonth = keyCountRelativePerMonth.values.reshape((-1,12))
# The group by causes Nov-Dez 2021 to occur, remove them.
keyCountPerMonth = keyCountPerMonth[:-2]

ax = sns.heatmap(keyCountPerMonth.T, vmin=0, yticklabels=['C', 'C#/Db', 'D', 'D#/Eb', 'E', 'F', 'F#/Gb', 'G', 'G#/Ab', 'A', 'A#/Bb', 'B'], xticklabels=xlabels_clean)
# Set the y labels to be not rotated
ax.set_yticklabels(ax.get_yticklabels(), rotation=0)

# Show colorbar with percentages
cbar = ax.collections[0].colorbar
cbar.set_ticks([0, .05, .1, .15, .2, .223])
cbar.set_ticklabels(['0%', '5%', '10%', '15%', '20%', '22.3%'])

In [35]:
modeCounts = merged_table.groupby(by=['year', 'month', 'mode'])['id'].count()
modeCountRelativePerMonth = modeCounts.groupby(by=['year', 'month']).transform(lambda x: x / max(x.sum(), 1))
modeCountPerMonth = modeCountRelativePerMonth.values.reshape((-1,2))
modeCountPerMonth = modeCountPerMonth[:-2]
merged_table['year_month'] = merged_table['year'].astype(str) + merged_table['month'].astype(str)
sns.displot(merged_table, x="year_month", hue="mode", multiple="dodge")
# keyCountPerMonth

In [44]:
sns.boxplot(x=merged_table['year'], y=merged_table['energy'])

In [45]:
chart = sns.boxplot(x=merged_table[merged_table['year'] >= 2020]['month'], y=merged_table['energy'])
# chart.set_xticklabels(chart.getxticklabels(), rotation=90)

In [46]:
chart = sns.boxplot(x=(merged_table['year'].astype(str) + '-' + merged_table['month'].astype(str)), y=merged_table['energy'])
chart.set_xticklabels(xlabels_clean, rotation=90)
chart.axvline(11.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(23.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(35.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(47.5, 0,1, color='r', linestyle='--', lw=2)
chart

In [47]:
sns.boxplot(x=merged_table['year'], y=merged_table['valence'])

In [48]:
sns.boxplot(x=merged_table['month'], y=merged_table['valence'])

In [49]:
chart = sns.boxplot(x=(merged_table['year'].astype(str) + '-' + merged_table['month'].astype(str)), y=merged_table['valence'])
chart.set_xticklabels(xlabels_clean, rotation=90)
chart.axvline(11.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(23.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(35.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(47.5, 0,1, color='r', linestyle='--', lw=2)
chart

In [50]:
sns.boxplot(x=merged_table['year'], y=merged_table['liveness'])

In [51]:
sns.boxplot(x=merged_table['month'], y=merged_table['liveness'])

In [52]:
chart = sns.boxplot(x=(merged_table['year'].astype(str) + '-' + merged_table['month'].astype(str)), y=merged_table['liveness'])
chart.set_xticklabels(xlabels_clean, rotation=90)
chart.axvline(11.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(23.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(35.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(47.5, 0,1, color='r', linestyle='--', lw=2)
chart

In [54]:
sns.boxplot(x=merged_table['year'], y=merged_table['loudness'])

In [55]:
sns.boxplot(x=merged_table['month'], y=merged_table['loudness'])

In [53]:
chart = sns.boxplot(x=(merged_table['year'].astype(str) + '-' + merged_table['month'].astype(str)), y=merged_table['loudness'])
chart.set_xticklabels(xlabels_clean, rotation=90)
chart.axvline(11.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(23.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(35.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(47.5, 0,1, color='r', linestyle='--', lw=2)
chart

In [58]:
sns.boxplot(x=merged_table['year'], y=merged_table['danceability'])

In [59]:
sns.boxplot(x=merged_table['month'], y=merged_table['danceability'])

In [57]:
chart = sns.boxplot(x=(merged_table['year'].astype(str) + '-' + merged_table['month'].astype(str)), y=merged_table['danceability'])
chart.set_xticklabels(xlabels_clean, rotation=90)
chart.axvline(11.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(23.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(35.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(47.5, 0,1, color='r', linestyle='--', lw=2)
chart

In [63]:
sns.boxplot(x=merged_table['year'], y=merged_table['speechiness'])

In [62]:
sns.boxplot(x=merged_table['month'], y=merged_table['speechiness'])

In [60]:
chart = sns.boxplot(x=(merged_table['year'].astype(str) + '-' + merged_table['month'].astype(str)), y=merged_table['speechiness'])
chart.set_xticklabels(xlabels_clean, rotation=90)
chart.axvline(11.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(23.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(35.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(47.5, 0,1, color='r', linestyle='--', lw=2)
chart

In [66]:
sns.boxplot(x=merged_table['year'], y=merged_table['tempo'])

In [65]:
sns.boxplot(x=merged_table['month'], y=merged_table['tempo'])

In [64]:
chart = sns.boxplot(x=(merged_table['year'].astype(str) + '-' + merged_table['month'].astype(str)), y=merged_table['tempo'])
chart.set_xticklabels(xlabels_clean, rotation=90)
chart.axvline(11.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(23.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(35.5, 0,1, color='r', linestyle='--', lw=2)
chart.axvline(47.5, 0,1, color='r', linestyle='--', lw=2)
chart